In [ ]:
# Recently FastAI is updated to version v2 so this code will update you on what has been changed
# remember in kaggle you have to turn internet to download the new packages (settings->internet(on))

# Importing the train and test csv files which contain the information whether the image is representing emergency vehicle or not

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
import fastai; print(fastai.__version__)

In [ ]:
from fastai.vision.all import *

In [ ]:
path = '../input/hackerearths-snakes-in-the-hood/dataset'
train_data = pd.read_csv('../input/hackerearths-snakes-in-the-hood/dataset/train.csv')
test_data = pd.read_csv('../input/hackerearths-snakes-in-the-hood/dataset/test.csv')

In [ ]:
train_data['image_id'] = train_data['image_id'].apply(lambda x:x+'.jpg')

In [ ]:
test_data['image_id'] = test_data['image_id'].apply(lambda x:x+'.jpg')

In [ ]:
train_data.head()

# Performing basic Augmentations

In [ ]:
# this will add flip, warp, zoom and rotation to the images
# tfms = aug_transforms(do_flip = True, max_lighting = 0.2, max_zoom= 1.1, max_warp = 0.15, max_rotate = 45)
item_tfms = [Resize(224, method='crop')]
batch_tfms=[*aug_transforms( do_flip=True,flip_vert=False ,max_rotate=20.0, max_zoom=1.3, max_lighting=0.5, max_warp=0.1, p_affine=0.2,
                      p_lighting=0.55), Normalize.from_stats(*imagenet_stats)]

In [ ]:
# batch_tfms = [*tfms,Normalize.from_stats(*imagenet_stats)]

In [ ]:
from sklearn.model_selection import StratifiedKFold
X,y = train_data['image_id'], train_data['breed']

# Resnet101 

In [ ]:
import torch

In [ ]:
predictions = np.zeros((test_data.shape[0], 35))
oof = np.zeros(train_data.shape[0])

In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
# mapping_label_breed = dict(zip(train_data['breed'].unique(), range(len(train_data['breed'].unique()))))

In [ ]:
# inv_mapping_label_breed = dict(zip(range(len(train_data['breed'].unique())), train_data['breed'].unique()))

In [ ]:
# train_data['breed'] = train_data['breed'].map(mapping_label_breed)

In [ ]:
!pip install pytorchcv torch>=0.4.0
from pytorchcv.model_provider import get_model as ptcv_get_model

In [ ]:
def shufflenet(pretrained=True):
    net = ptcv_get_model("squeezenet1_1", pretrained=True)
    net2 = nn.Sequential()
    net2.add_module("features", net.features)
#     net2.add_module("features", net.features[:-1])

    net2.add_module("final_pool", nn.AdaptiveAvgPool2d(output_size=1))
    net2.add_module("fc", nn.Linear(in_features=4096, out_features=35))
    return net2

In [ ]:
# we will be using resnet 101 model

all_models = [models.resnet101]

fnames = get_image_files('../input/hackerearths-snakes-in-the-hood/dataset/test')

for model in all_models:
    for fold, (tr_ids, val_ids) in enumerate(StratifiedKFold(n_splits=5).split(X, y)):
        import gc
        gc.collect()
        torch.cuda.empty_cache()
        train_data.loc[val_ids,'valid'] = 1
        train_data.loc[tr_ids,'valid'] = 0
        
        snake_breed_dls = ImageDataLoaders.from_df(train_data,path+'/train/', 
                              batch_tfms = batch_tfms,
                              bs = 64,
                              val_bs=64,
                              item_tfms=item_tfms,
                              valid_col = 'valid',
                              fn_col='image_id',
                              label_col='breed')
        
        print(snake_breed_dls.valid_ds.__len__())
        
        
        learn = cnn_learner(snake_breed_dls, model,metrics=[CohenKappa(), F1Score(average='weighted'),accuracy], model_dir="/tmp/model/",wd=1e-05)
        
        
        learn.save('/tmp/model/'+model.__name__)
        
        sbs=SaveModelCallback(monitor='cohen_kappa_score', fname=model.__name__)
        
        torch.cuda.set_device(0)
        learn.unfreeze()
        learn.fit_one_cycle(25, cbs=[sbs, ])
        
        learn = learn.load('/tmp/model/'+model.__name__)
       
        dl = learn.dls.test_dl(fnames)
        
        res = learn.tta( dl=dl,n=8)
        
        predictions += np.array(res[0])
        

In [ ]:
category = {'agkistrodon-contortrix':1,
            'agkistrodon-piscivorus':2, 
            'coluber-constrictor':3, 
            'crotalus-atrox':4, 
            'crotalus-horridus':5, 
            'crotalus-ruber':6, 
            'crotalus-scutulatus':7, 
            'crotalus-viridis':8, 
            'diadophis-punctatus':9, 
            'haldea-striatula':10, 
            'heterodon-platirhinos':11, 
            'lampropeltis-californiae':12, 
            'lampropeltis-triangulum':13, 
            'masticophis-flagellum':14, 
            'natrix-natrix':15, 
            'nerodia-erythrogaster':16, 
            'nerodia-fasciata':17, 
            'nerodia-rhombifer':18, 
            'nerodia-sipedon':19, 
            'opheodrys-aestivus':20, 
            'pantherophis-alleghaniensis':21, 
            'pantherophis-emoryi':22, 
            'pantherophis-guttatus':23, 
            'pantherophis-obsoletus':24, 
            'pantherophis-spiloides':25, 
            'pantherophis-vulpinus':26, 
            'pituophis-catenifer':27, 
            'rhinocheilus-lecontei':28, 
            'storeria-dekayi':29, 
            'storeria-occipitomaculata':30, 
            'thamnophis-elegans':31, 
            'thamnophis-marcianus':32, 
            'thamnophis-proximus':33, 
            'thamnophis-radix':34, 
            'thamnophis-sirtalis':35}


rev_category = {val: key for key, val in category.items()}

In [ ]:
final_submit = pd.DataFrame(columns=['image_id', 'breed'])
final_submit['image_id'] = [str(i).split('/')[5].split('.')[0] for i in fnames]
final_submit['breed'] = np.argmax(predictions, axis=1)+1
final_submit['breed'] = final_submit['breed'].map(rev_category)
final_submit.to_csv('submit.csv', index=False)

In [ ]:
final_submit.head()